In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import quandl
from datetime import datetime
from dateutil.parser import parse
from itertools import chain
import operator
import sys
from fbprophet import Prophet

In [2]:
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

In [4]:
btc_data = pd.read_csv("data_final.csv")
btc_data.set_index('Date', inplace=True)
btc_data.head()

,open,high,low,close,weighted_price,high_low_spread,midquote_price,effective_spread,7_trailing,7_next_trailing,volume_btc,volume_currency
Date,,,,,,,,,,,,
2016-01-01,431.35,435.88,428.22,433.66,432.87,7.66,3.83,859.65,-22.526170,20.216172,8749.673805,3789078.83
2016-01-02,434.71,435.99,431.18,434.44,433.70,4.81,2.41,864.07,16.345000,15.412500,6591.310053,2862465.49
2016-01-03,434.02,434.53,424.06,429.88,429.01,10.47,5.24,849.28,5.965002,19.124997,9678.402186,4157151.63
2016-01-04,430.43,435.12,428.82,433.01,432.40,6.30,3.15,859.72,10.813537,16.589110,13463.971066,5832898.79
2016-01-05,432.75,435.14,429.50,432.35,432.15,5.64,2.82,859.06,0.085000,3.425000,10381.184703,4492484.21


In [7]:
btc_data['y'] = np.log(btc_data['close'])
btc_data['ds'] = btc_data.index
fb_reg_data = btc_data.loc[:,['ds','y']]
fb_reg_data = fb_reg_data.reset_index(drop=True)
fb_reg_data.dropna(inplace=True)
fb_reg_data.head()

,ds,y
0,2016-01-01,6.072261
1,2016-01-02,6.074058
2,2016-01-03,6.063506
3,2016-01-04,6.070761
4,2016-01-05,6.069235


In [8]:
from fbprophet import Prophet
import seaborn as sns
import matplotlib.dates as mdates

In [9]:
def plot_data(df,X,future,columns,title):

# lot_data(predicted, actual, future, cols,title)
    trace1 = go.Scatter(
        x=future['ds'],
        y=X,
        name='Actual Close'
    )
    #Lower_closing	Upper_closing
    trace2 = go.Scatter(
        x=future['ds'],
        y=df['Upper_closing'],
        name='Predicted Close Upper Bound',
        fill=None,
        mode='lines',
        line=dict(color='rgb(143,19,131)'),        
       
    )
    trace3 = go.Scatter(
        x=future['ds'],
        y=df['Lower_closing'],
        name='Predicted Close Lower Bound',
        fill='tonexty',
        mode='lines',
        line=dict(color='rgb(143,19,131)'),
    )

    data = [trace1, trace2, trace3]
    layout = go.Layout(
        title='Actual Vs Predicted using FBProphet',
        yaxis=dict(
            title='Closing Price'
        ),
        yaxis2=dict(
            title='yaxis2 title',
            titlefont=dict(
                color='rgb(148, 103, 189)'
            ),
            tickfont=dict(
                color='rgb(148, 103, 189)'
            ),
            overlaying='y',
            side='right'
        )
    )
    fig = go.Figure(data=data, layout=layout)
    #plot_url = py.plot(fig)
    py.iplot(fig)
    #plt.show()

In [15]:
fb_reg_data_new = fb_reg_data[639:]
fb_reg_data_new.head()

,ds,y
639,2017-10-01,8.386082
640,2017-10-02,8.387886
641,2017-10-03,8.368904
642,2017-10-04,8.347240
643,2017-10-05,8.369697


In [22]:
m = Prophet(changepoint_prior_scale = 0.5)
m.fit(fb_reg_data_new)
future = m.make_future_dataframe(periods=3)
future.tail()

C:\Users\jamie\Anaconda3\envs\PythonData\lib\site-packages\pystan\misc.py:399: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.



,ds
90,2017-12-30
91,2017-12-31
92,2018-01-01
93,2018-01-02
94,2018-01-03


In [23]:

forecast = m.predict(future)
predicted  = forecast[['yhat', 'yhat_lower', 'yhat_upper']].applymap(np.exp)
cols = ["Mean", "Lower_closing", "Upper_closing"]
predicted.columns = cols
actual = np.exp(fb_reg_data_new["y"])
actual.name = "Actual"
title = "Closing Price Distribution of Bitcoin"
plot_data(predicted, actual, future, cols,title)

In [24]:
btc_data.head()

,open,high,low,close,weighted_price,high_low_spread,midquote_price,effective_spread,7_trailing,7_next_trailing,volume_btc,volume_currency,y,ds
Date,,,,,,,,,,,,,,
2016-01-01,431.35,435.88,428.22,433.66,432.87,7.66,3.83,859.65,-22.526170,20.216172,8749.673805,3789078.83,6.072261,2016-01-01
2016-01-02,434.71,435.99,431.18,434.44,433.70,4.81,2.41,864.07,16.345000,15.412500,6591.310053,2862465.49,6.074058,2016-01-02
2016-01-03,434.02,434.53,424.06,429.88,429.01,10.47,5.24,849.28,5.965002,19.124997,9678.402186,4157151.63,6.063506,2016-01-03
2016-01-04,430.43,435.12,428.82,433.01,432.40,6.30,3.15,859.72,10.813537,16.589110,13463.971066,5832898.79,6.070761,2016-01-04
2016-01-05,432.75,435.14,429.50,432.35,432.15,5.64,2.82,859.06,0.085000,3.425000,10381.184703,4492484.21,6.069235,2016-01-05


In [25]:
btc_data.reset_index(inplace=True)

In [26]:
btc_data.head()

,Date,open,high,low,close,weighted_price,high_low_spread,midquote_price,effective_spread,7_trailing,7_next_trailing,volume_btc,volume_currency,y,ds
0,2016-01-01,431.35,435.88,428.22,433.66,432.87,7.66,3.83,859.65,-22.526170,20.216172,8749.673805,3789078.83,6.072261,2016-01-01
1,2016-01-02,434.71,435.99,431.18,434.44,433.70,4.81,2.41,864.07,16.345000,15.412500,6591.310053,2862465.49,6.074058,2016-01-02
2,2016-01-03,434.02,434.53,424.06,429.88,429.01,10.47,5.24,849.28,5.965002,19.124997,9678.402186,4157151.63,6.063506,2016-01-03
3,2016-01-04,430.43,435.12,428.82,433.01,432.40,6.30,3.15,859.72,10.813537,16.589110,13463.971066,5832898.79,6.070761,2016-01-04
4,2016-01-05,432.75,435.14,429.50,432.35,432.15,5.64,2.82,859.06,0.085000,3.425000,10381.184703,4492484.21,6.069235,2016-01-05


In [53]:
import datetime
import numpy as np
def date_parse(date_string):
    date_list = date_string.split('-')
    date_index_0 = datetime.date(2016, 1, 1)
    date = datetime.date(int(date_list[0]),int(date_list[1]),int(date_list[2]))
    diff = date - date_index_0
    return diff.days

In [54]:
date_string = "2017-10-1"

td = date_parse(date_string)
td

639